# Interact with SQLite and DuckDB

In [ ]:
!pip install duckdb

In [ ]:
import sqlite3

import duckdb
import pandas as pd

In [ ]:
# create connections for sqlite and duckdb
generator_duckdb_conn = duckdb.connect(database=":memory:")
duckdb_conn = duckdb.connect(database="example.duckdb")
sqlite_conn = sqlite3.connect("example.sqlite")

In [ ]:
# generate synthetic data
generator_duckdb_conn.execute("CALL dbgen(sf=0.1);")

In [ ]:
generator_duckdb_conn.execute("show tables;").df()

In [ ]:
generator_duckdb_conn.execute("describe orders;").df()

In [ ]:
generator_duckdb_conn.execute("select * from orders;").df()

In [ ]:
# export tables from generator duckdb in-mem database to sqlite database
for table in generator_duckdb_conn.execute("show tables;").df()["name"].values.tolist():
    print(f"Exporting {table} table data from duckdb generator connection to sqlite")
    generator_duckdb_conn.execute(f"select * from {table};").df().to_sql(
        name=table, con=sqlite_conn, if_exists="replace", index=False
    )

In [ ]:
# close generator connection
generator_duckdb_conn.close()

In [ ]:
pd.read_sql(
    sql="SELECT name FROM sqlite_schema WHERE type ='table'", con=sqlite_conn
).values.tolist()

In [ ]:
# export data from sqlite tables to duckdb database
for table in pd.read_sql(
    sql="SELECT name FROM sqlite_schema WHERE type ='table'", con=sqlite_conn
)["name"].values.tolist():
    df = pd.read_sql(sql=f"select * from {table}", con=sqlite_conn)
    duckdb_conn.execute(f"CREATE TABLE IF NOT EXISTS {table} AS SELECT * FROM df")

In [ ]:
duckdb_conn.execute("select * from orders;").df()

In [ ]:
# close connections
sqlite_conn.close()
duckdb_conn.close()

In [ ]:
# show relative filesizes
!ls -lh ./ | egrep "\.(duckdb|sqlite)"